# Requirements

In [ ]:
from math import isnan

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import wandb

dataset_match = {
    "fbirn": "FBIRN ICA",
    "fbirn_roi": "FBIRN Schaefer 200 ROI",
    "bsnip": "BSNIP ICA",
    "cobre": "COBRE ICA",

    "abide": "ABIDE ICA 569 subjects",
    "abide_869": "ABIDE ICA",
    "abide_roi": "ABIDE Schaefer 200 ROI",

    "oasis": "OASIS ICA",
    "adni": "ADNI ICA",

    "hcp": "HCP ICA",
    "hcp_roi": "HCP Schaefer 200 ROI Extended",
    "hcp_roi_752": "HCP Schaefer 200 ROI",
    "hcp_schaefer": "HCP Schaefer 200 ROI w/o denoising",
    "hcp_non_mni_2": "HCP Deskian/Killiany ROI in Orig space",
    "hcp_mni_3": "HCP Deskian/Killiany ROI",

    "ukb": "UK Biobank (Sex) ICA",
    "ukb_age_bins": "UK Biobank (Age-Sex) ICA",

    "hcp_time": "HCP ICA Time",

    "fbirn_cobre": "FBIRN X COBRE",
    "fbirn_bsnip": "FBIRN X BSNIP",
    "cobre_fbirn": "COBRE X FBIRN",
    "cobre_bsnip": "COBRE X BSNIP",
    "bsnip_fbirn": "BSNIP X FBIRN",
    "bsnip_cobre": "BSNIP X COBRE",

    "oasis_adni": "OASIS X ADNI",
    "adni_oasis": "ADNI X OASIS",
}

model_match = {
    "mlp": "Old Mean MLP",
    "rearranged_mlp": "Mean MLP",
    "attn_mlp": "Attention MLP",
    "lstm": "LSTM",
    "mean_lstm": "Mean LSTM",
    "transformer": "Transformer w/o PE",
    "mean_transformer": "Mean Transformer w/o PE",
    "pe_transformer": "Transformer",
    "mean_pe_transformer": "Mean Transformer",
    "milc": "MILC",
    "dice": "DICE",
    "bnt": "BNT",
    "fbnetgen": "FBNetGen",
    "brainnetcnn": "BrainNetCNN",
    "lr": "LR",
}

def fix_run(run, dataset):
    for _, row in run.history().iterrows():
        if f"{dataset}score" in row and f"{dataset}accuracy" in row and "training_time" in row and f"{dataset}average_time" in row:
            if row[f"{dataset}score"] is not None and row[f"{dataset}accuracy"] is not None and row["training_time"] is not None and row[f"{dataset}average_time"] is not None:
                if not isnan(row[f"{dataset}score"]) and not isnan(row[f"{dataset}accuracy"]) and not isnan(row["training_time"]) and not isnan(row[f"{dataset}average_time"]):
                    auc = row[f"{dataset}score"]
                    acc = row[f"{dataset}accuracy"]
                    train_t = row["training_time"]
                    inf_t= row[f"{dataset}average_time"]
                    break
    
    run.summary[f"{dataset}score"] = auc
    run.summary[f"{dataset}accuracy"] = acc
    run.summary["training_time"] = train_t
    run.summary[f"{dataset}average_time"] = inf_t

    run.summary.update()

    return auc, acc, train_t, inf_t

def load_run(proj_name, dataset):
    if dataset is not None:
        dataset += "_"
    else:
        dataset = "test_"

    api = wandb.Api(timeout=19)
    runs = api.runs(f"pavalipopov/{proj_name}")

    summary_list = []
    runs_list = []
    config_list = []
    for i, run in enumerate(runs):
        summary_list.append(run.summary._json_dict)
        config_list.append(run.config)
        runs_list.append(run)


    AUC_score = []
    accuracy = []
    train_time = []
    inference_time = []
    params = []
    shuffling = []

    for i, summary in enumerate(summary_list):
        everything_is_cool = True
        everything_is_cool = everything_is_cool and f"{dataset}score" in summary and summary[f"{dataset}score"] is not None
        everything_is_cool = everything_is_cool and f"{dataset}accuracy" in summary and summary[f"{dataset}accuracy"] is not None
        everything_is_cool = everything_is_cool and "training_time" in summary and summary["training_time"] is not None
        everything_is_cool = everything_is_cool and f"{dataset}average_time" in summary and summary[f"{dataset}average_time"] is not None

        if everything_is_cool:
            AUC_score.append(summary[f"{dataset}score"])
            accuracy.append(summary[f"{dataset}accuracy"])
            train_time.append(summary["training_time"])
            inference_time.append(summary[f"{dataset}average_time"])
        else:
            print(f"Run {i} summary is broken, fixing it")
            auc, acc, train_t, inf_t = fix_run(runs_list[i], dataset)
            AUC_score.append(auc)
            accuracy.append(acc)
            train_time.append(train_t)
            inference_time.append(inf_t)

        if "params" in summary:
            params.append(summary["params"])
        else:
            params.append(None)

        if "permute" in config_list[i]["general"]:
            if config_list[i]["general"]["permute"] == False:
                shuffling.append("None")
            else:
                shuffling.append(config_list[i]["general"]["permute"])
        else:
            shuffling.append("None")
    
    return AUC_score, accuracy, train_time, inference_time, params, shuffling

def load_metrics(paths_dict, ds_dict, model_dict):
    data_list = []

    for model_name in paths_dict:
        print(model_name)
        for dataset_name in paths_dict[model_name]:
            print("\t ", dataset_name)
            
            if len(paths_dict[model_name][dataset_name]) == 2:
                path = paths_dict[model_name][dataset_name][0]
                dataset = paths_dict[model_name][dataset_name][1]
            elif len(paths_dict[model_name][dataset_name]) == 1:
                path = paths_dict[model_name][dataset_name][0]
                dataset = "test"
            else:
                raise NotImplementedError()

            print("\t\t sub_dataset:", dataset)

            AUC_score, accuracy, train_time, inference_time, params, shuffling = load_run(path, dataset)
            
            if dataset != "test":
                dataset = dataset_name + "_" + dataset
            else:
                dataset = dataset_name
            length = len(AUC_score)

            data_list.append(
                pd.DataFrame(
                    {
                        "Model": [model_dict[model_name]]*length,
                        "Dataset": [ds_dict[dataset]]*length,
                        "AUROC": AUC_score,
                        "Accuracy": accuracy,
                        "Train time": train_time,
                        "Inference time": inference_time,
                        "Trainable params": params,
                        "Shuffling": shuffling,
                    }
                )
            )
    
    return pd.concat(data_list)


In [ ]:
def test_run(proj_name, dataset):
    if dataset is not None:
        dataset += "_"
    else:
        dataset = "test_"

    api = wandb.Api(timeout=19)
    # Project is specified by <entity/project-name>

    try:
        runs = api.runs(f"pavalipopov/{proj_name}")
        summary_list = []
        runs_list = []
        config_list = []
        for i, run in enumerate(runs): 
            # print(f"Run {i}")
            # .summary contains the output keys/values for metrics like accuracy.
            #  We call ._json_dict to omit large files 
            summary_list.append(run.summary._json_dict)
            config_list.append(run.config)
            runs_list.append(run)
    except:
        print(proj_name)
        return

    global_cool = True

    count = 0
    for i, summary in enumerate(summary_list):
        everything_is_cool = True
        everything_is_cool = everything_is_cool and f"{dataset}score" in summary and summary[f"{dataset}score"] is not None
        everything_is_cool = everything_is_cool and f"{dataset}accuracy" in summary and summary[f"{dataset}accuracy"] is not None
        everything_is_cool = everything_is_cool and "training_time" in summary and summary["training_time"] is not None
        everything_is_cool = everything_is_cool and f"{dataset}average_time" in summary and summary[f"{dataset}average_time"] is not None

        if not everything_is_cool:
            try:
                fix_run(runs_list[i], dataset)
                count += 1
            except:
                global_cool = False
            # print(f"Run {i} summary is broken")
        else:
            count += 1
    
    if not global_cool or count != 50:
        print(proj_name)

def test_runs(paths_dict):

    print("Problematic runs:")

    for model_name in paths_dict:
        # print(model_name)
        for dataset_name in paths_dict[model_name]:
            # print("\t ", dataset_name)
            
            if len(paths_dict[model_name][dataset_name]) == 2:
                path = paths_dict[model_name][dataset_name][0]
                dataset = paths_dict[model_name][dataset_name][1]
            elif len(paths_dict[model_name][dataset_name]) == 1:
                path = paths_dict[model_name][dataset_name][0]
                dataset = "test"
            else:
                raise NotImplementedError()

            # print("\t\t sub_dataset:", dataset)

            test_run(path, dataset)

# Load data

## General results

In [ ]:
datasets = [
    "fbirn", 
    "fbirn_roi",
    "bsnip",
    "cobre",
    "abide_869",
    "abide_roi",
    "oasis",
    "adni",
    "hcp",
    "hcp_roi_752",
    "ukb",
    "ukb_age_bins",
]
dataset_suffix = "_th"
models = [
    # "mlp",
    "lstm",
    # "transformer",
    "milc",
    "dice",
    "bnt",
    "fbnetgen",
    "brainnetcnn",
    "lr",
]
model_suffix = "_defHP"

prefix = "rerun_all"
suffix = ""

general_projects = {}
for model in models:
    general_projects[model] = {}

    project_model = model
    if model_suffix is not None:
        project_model += model_suffix

    for dataset in datasets:
        project_dataset = dataset
        if dataset in ["ukb", "ukb_age_bins"]:
            project_dataset += dataset_suffix

        general_projects[model][dataset] = (f"{prefix}-exp-{project_model}-{project_dataset}{suffix}", )
        if model == "lr" and dataset in ["fbirn", "cobre", "bsnip", "oasis", "adni"]:
            general_projects[model][dataset] = (f"cross_ds-exp-{project_model}-{project_dataset}{suffix}", )

models = [
    "rearranged_mlp",
    "mean_lstm",
    # "mean_transformer",
    "pe_transformer",
    "mean_pe_transformer",
]
prefix = "rerun_all2"
for model in models:
    general_projects[model] = {}

    project_model = model
    if model_suffix is not None:
        project_model += model_suffix

    for dataset in datasets:
        project_dataset = dataset
        if dataset in ["ukb", "ukb_age_bins"]:
            project_dataset += dataset_suffix

        general_projects[model][dataset] = (f"{prefix}-exp-{project_model}-{project_dataset}{suffix}", )

In [ ]:
comparison_data = load_metrics(general_projects, dataset_match, model_match)

# Plot data

## Model Comparisons

In [ ]:
plot_data = comparison_data.query(
        "Shuffling == 'None'")

models = [
    "Mean MLP",
    'LSTM',
    'Transformer',
    # 'Mean LSTM',
    # 'Mean Transformer',
    # 'PE Transformer',
    # 'Mean PE Transformer',
    "MILC", 
    "DICE", 
    "BNT", 
    "FBNetGen", 
    "BrainNetCNN", 
    "LR"
]
datasets = [
    "FBIRN ICA",
    "BSNIP ICA",
    "COBRE ICA",
    "ABIDE ICA",
    "OASIS ICA",
    "ADNI ICA",
    "HCP ICA",
    "UK Biobank (Sex) ICA",
    "UK Biobank (Age-Sex) ICA",

    "FBIRN Schaefer 200 ROI",
    "ABIDE Schaefer 200 ROI",
    "HCP Schaefer 200 ROI",

    "HCP ICA Time",
]

rename_dict = {
    "FBIRN ICA": "FBIRN",
    "BSNIP ICA": "BSNIP",
    "COBRE ICA": "COBRE",
    "ABIDE ICA": "ABIDE",
    "OASIS ICA": "OASIS",
    "ADNI ICA": "ADNI",
    "HCP ICA": "HCP",
    "UK Biobank (Sex) ICA": "UKB-S",
    "UK Biobank (Age-Sex) ICA": "UKB-SA",

    "FBIRN Schaefer 200 ROI": "FBIRN",
    "ABIDE Schaefer 200 ROI": "ABIDE",
    "HCP Schaefer 200 ROI": "HCP",

    "HCP ICA Time": "HCP",
}

hue_order = models

ica_datasets = datasets[:7]
data_1 = plot_data.query("Dataset in @ica_datasets")
data_1["Dataset"] = data_1["Dataset"].replace(rename_dict)
ica_datasets = [rename_dict[dataset] for dataset in ica_datasets]

ukb_s_datasets = datasets[7:8]
data_s_ukb = plot_data.query("Dataset in @ukb_s_datasets")
data_s_ukb["Dataset"] = data_s_ukb["Dataset"].replace(rename_dict)
ukb_s_datasets = [rename_dict[dataset] for dataset in ukb_s_datasets]

ukb_sa_datasets = datasets[8:9]
data_sa_ukb = plot_data.query("Dataset in @ukb_sa_datasets")
data_sa_ukb["Dataset"] = data_sa_ukb["Dataset"].replace(rename_dict)
ukb_sa_datasets = [rename_dict[dataset] for dataset in ukb_sa_datasets]

roi_datasets = datasets[9:12]
data_2 = plot_data.query("Dataset in @roi_datasets")
data_2["Dataset"] = data_2["Dataset"].replace(rename_dict)
roi_datasets = [rename_dict[dataset] for dataset in roi_datasets]

time_datasets = datasets[12:]
data_3 = plot_data.query("Dataset in @time_datasets")
data_3["Dataset"] = data_3["Dataset"].replace(rename_dict)
time_datasets = [rename_dict[dataset] for dataset in time_datasets]

palette = { item: plt.cm.tab20(i) for i, item in enumerate(models)}

In [ ]:
import matplotlib.ticker as ticker
from statannot import add_stat_annotation


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sns.reset_orig()
sns.set_theme(
    style="whitegrid", 
    font_scale = 1.5,
)

# Create a figure and a grid of subplots (2 rows, 1 column)
fig = plt.figure(figsize=(18, 10))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1], hspace=0.3)

# Create subplots for each subfigure
subfig11 = plt.subplot(gs[0])

# Divide the lower row into two subplots with different widths
# subgs2 = gridspec.GridSpecFromSubplotSpec(1, 4, subplot_spec=gs[1], width_ratios=[1, 1, 3, 2])
subgs2 = gridspec.GridSpecFromSubplotSpec(1, 4, subplot_spec=gs[1], width_ratios=[2, 2, 6, 2])
subfig12 = plt.subplot(subgs2[0])
subfig13 = plt.subplot(subgs2[1])
subfig2 = plt.subplot(subgs2[2])
subfig3 = plt.subplot(subgs2[3])

### add ghost figure for UKB results title
ghostsubgs2 = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=gs[1], width_ratios=[4, 6, 2])
ghost_ax = fig.add_subplot(ghostsubgs2[0])
ghost_ax.axis('off')
ghost_ax.set_title('ICA - Neuromark')
###### plot data

## row 1
boxplot_1 = sns.boxplot(
    x="Dataset", 
    order=ica_datasets,
    y="AUROC",
    hue="Model",
    hue_order=hue_order,
    data=data_1,
    palette=palette,
    showfliers = False,
    ax=subfig11,
)
subfig11.legend_.remove()
subfig11.axhline(y=0.5, linestyle='dashed')
subfig11.set_xlabel(None)
subfig11.set_yticks([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
subfig11.set_ylim(0.45, 1.05)
subfig11.set_ylabel("ROC AUC")
subfig11.set_title('ICA - Neuromark')

## row 2
boxplot_2 = sns.boxplot(
    x="Dataset", 
    order=ukb_s_datasets,
    y="AUROC",
    hue="Model",
    hue_order=hue_order,
    data=data_s_ukb,
    palette=palette,
    showfliers = False,
    ax=subfig12,
)
subfig12.legend_.remove()
subfig12.axhline(y=0.5, linestyle='dashed')
subfig12.set_xlabel(None)
# subfig11.set_yticks([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
subfig12.set_ylim(0.935, 0.975)
subfig12.set_ylabel("ROC AUC")


boxplot_3 = sns.boxplot(
    x="Dataset", 
    order=ukb_sa_datasets,
    y="AUROC",
    hue="Model",
    hue_order=hue_order,
    data=data_sa_ukb,
    palette=palette,
    showfliers = False,
    ax=subfig13,
)
subfig13.legend_.remove()
subfig13.axhline(y=0.5, linestyle='dashed')
subfig13.set_xlabel(None)
# subfig13.set_yticks([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
subfig13.set_ylim(0.745, 0.855)
subfig13.set_ylabel(None)

boxplot_4 = sns.boxplot(
    x="Dataset", 
    order=roi_datasets,
    y="AUROC",
    hue="Model",
    hue_order=hue_order,
    data=data_2,
    palette=palette,
    showfliers = False,
    ax=subfig2,
)
subfig2.set_title('ROI - Schaefer 200 ROIs atlas')
subfig2.legend_.remove()
subfig2.axhline(y=0.5, linestyle='dashed')
subfig2.set_xlabel(None)
subfig2.set_yticks([0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
subfig2.set_ylim(0.45, 1.05)
subfig2.set_ylabel(None)

###### Add hatches

for i in range(10, 18):
    boxplot_1.patches[i].set_hatch("///")
for i, box in enumerate(boxplot_1.patches):
    if i > 18:
        if i % 9 == 5 or i % 9 == 6 or i % 9 == 7 or i % 9 == 8:
            box.set_hatch("///")

for i in range(10, 18):
    boxplot_2.patches[i].set_hatch("///")
for i, box in enumerate(boxplot_2.patches):
    if i > 18:
        if i % 9 == 5 or i % 9 == 6 or i % 9 == 7 or i % 9 == 8:
            box.set_hatch("///")

for i in range(10, 18):
    boxplot_3.patches[i].set_hatch("///")
for i, box in enumerate(boxplot_3.patches):
    if i > 18:
        if i % 9 == 5 or i % 9 == 6 or i % 9 == 7 or i % 9 == 8:
            box.set_hatch("///")

for i in range(10, 18):
    boxplot_4.patches[i].set_hatch("///")
for i, box in enumerate(boxplot_4.patches):
    if i > 18:
        if i % 9 == 5 or i % 9 == 6 or i % 9 == 7 or i % 9 == 8:
            box.set_hatch("///")

# Create a legend in subfigure 3 for the boxplots
subfig3.axis('off')  # Turn off axis for legend subplot
legend_handles, legend_labels = subfig11.get_legend_handles_labels()

ts_models = [
    "Mean MLP", 
    'LSTM',
    'Transformer',
    "MILC", 
    "DICE", 
]
fnc_models = [
    "BNT", 
    "FBNetGen", 
    "BrainNetCNN", 
    "LR"
]

ts_handles = [handle for label, handle in zip(legend_labels, legend_handles) if label in ts_models]
ts_labels = [label for label in legend_labels if label in ts_models]
fnc_handles = [handle for label, handle in zip(legend_labels, legend_handles) if label in fnc_models]
fnc_labels = [label for label in legend_labels if label in fnc_models]

# Create legends for each category and place them in subfig4
legend_ts = subfig3.legend(handles=ts_handles, labels=ts_labels, title='Time Series Models', loc='upper left')

# # Calculate the position for legend_fnc to the right of legend_ts
legend_ts_bbox = legend_ts.get_bbox_to_anchor().transformed(subfig3.transAxes.inverted())
legend_fnc = subfig3.legend(handles=fnc_handles, labels=fnc_labels, title='FNC Models', loc='upper left', bbox_to_anchor=(legend_ts_bbox.x0, legend_ts_bbox.y0+0.35))
# legend_fnc = subfig3.legend(handles=fnc_handles, labels=fnc_labels, title='FNC Models', loc='upper left')

# # Readd the TS legend box to subfig3
subfig3.add_artist(legend_ts)

# legend = subfig3.legend(handles=legend_handles,
#                         labels=legend_labels, loc='right', title='Models')
# subfig3.add_artist(legend)

# # Adjust spacing between subplots
# plt.tight_layout()

### Add Wilcoxon tests

box_pairs = [((dataset, "Mean MLP"), (dataset, "DICE")) for dataset in ica_datasets]
box_pairs += [((dataset, "BNT"), (dataset, "LR")) for dataset in ica_datasets]
add_stat_annotation(subfig11, data=data_1, x="Dataset", y="AUROC", hue="Model", hue_order=hue_order,
                    box_pairs=box_pairs,
                    test='Wilcoxon', text_format='star', loc='inside', verbose=1, text_offset=-4, 
                    pvalue_thresholds = [[0.05, "*"], [1, u'\N{DEGREE SIGN}']],
                    line_offset=0.01, line_offset_to_box=0.01, line_height=0.01, fontsize=30)
subfig11.set_xlim(xmin=-0.5, xmax=6.5)
subfig11.set_ylim(0.45, 1.05)

box_pairs = [((dataset, "Mean MLP"), (dataset, "DICE")) for dataset in ukb_s_datasets]
box_pairs += [((dataset, "BNT"), (dataset, "LR")) for dataset in ukb_s_datasets]
add_stat_annotation(subfig12, data=data_s_ukb, x="Dataset", y="AUROC", hue="Model", hue_order=hue_order,
                    box_pairs=box_pairs,
                    test='Wilcoxon', text_format='star', loc='inside', verbose=1, text_offset=-4, 
                    pvalue_thresholds = [[0.05, "*"], [1, u'\N{DEGREE SIGN}']],
                    line_offset=0.01, line_offset_to_box=0.01, line_height=0.01, fontsize=30)
subfig12.set_xlim(xmin=-0.5, xmax=0.5)
subfig12.set_ylim(0.925, 0.98)

box_pairs = [((dataset, "Mean MLP"), (dataset, "DICE")) for dataset in ukb_sa_datasets]
box_pairs += [((dataset, "BNT"), (dataset, "LR")) for dataset in ukb_sa_datasets]
add_stat_annotation(subfig13, data=data_sa_ukb, x="Dataset", y="AUROC", hue="Model", hue_order=hue_order,
                    box_pairs=box_pairs,
                    test='Wilcoxon', text_format='star', loc='inside', verbose=1, text_offset=-4, 
                    pvalue_thresholds = [[0.05, "*"], [1, u'\N{DEGREE SIGN}']],
                    line_offset=0.01, line_offset_to_box=0.01, line_height=0.01, fontsize=30)
subfig13.set_xlim(xmin=-0.5, xmax=0.5)
subfig13.set_ylim(0.745, 0.845)

box_pairs = [((dataset, "Mean MLP"), (dataset, "DICE")) for dataset in roi_datasets]
box_pairs += [((dataset, "BNT"), (dataset, "LR")) for dataset in roi_datasets]
add_stat_annotation(subfig2, data=data_2, x="Dataset", y="AUROC", hue="Model", hue_order=hue_order,
                    box_pairs=box_pairs,
                    test='Wilcoxon', text_format='star', loc='inside', verbose=1, text_offset=-4, 
                    pvalue_thresholds = [[0.05, "*"], [1, u'\N{DEGREE SIGN}']],
                    line_offset=0.01, line_offset_to_box=0.01, line_height=0.01, fontsize=30)
subfig2.set_xlim(xmin=-0.5, xmax=2.5)
subfig2.set_ylim(0.45, 1.05)

# Show the plot
# plt.show()
plt.savefig(
    "general_comparison.png",
    format="png",
    # dpi=300,
    bbox_inches='tight',
)
# plt.savefig(
#     "general_comparison.svg",
#     format="svg",
#     # dpi=300,
#     bbox_inches='tight',
# )
plt.close()